In [1]:
# Import libraries # Load dataset

import pandas as pd
from pathlib import Path

# -----------------------------
# Project paths (single source)
# -----------------------------
PROJECT_ROOT = Path.cwd().parent
OUTPUTS_DIR = PROJECT_ROOT / "outputs"

# Safety check
assert OUTPUTS_DIR.exists(), f"Outputs directory not found: {OUTPUTS_DIR}"

# -----------------------------
# Load data
# -----------------------------
ndvi = pd.read_csv(OUTPUTS_DIR / "ndvi_lga_monthly_benue.csv")
soil = pd.read_csv(OUTPUTS_DIR / "soil_moisture_lga_monthly_benue.csv")
humidity = pd.read_csv(OUTPUTS_DIR / "humidity_lga_monthly_benue.csv")

ndvi.head()


,system:index,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,ADM2_CODE,ADM2_NAME,DISP_AREA,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng,date,mean,month,year,.geo
0,0_000000000000000010fb,182,Nigeria,2215,Benue,191192,Agatu,NO,3000,Member State,1999,0.102119,1.472957,2021-01,0.411941,1.0,2021.0,"{""type"":""Polygon"",""coordinates"":[[[7.665016316..."
1,0_00000000000000001107,182,Nigeria,2215,Benue,191197,Logo,NO,3000,Member State,1999,0.112918,1.671292,2021-01,0.323571,1.0,2021.0,"{""type"":""Polygon"",""coordinates"":[[[9.169418974..."
2,0_0000000000000000111a,182,Nigeria,2215,Benue,191204,Apa,NO,3000,Member State,1999,0.072915,1.070310,2021-01,0.419388,1.0,2021.0,"{""type"":""Polygon"",""coordinates"":[[[7.754657685..."
3,0_00000000000000001122,182,Nigeria,2215,Benue,191210,Tarka,NO,3000,Member State,1999,0.031404,0.750493,2021-01,0.369389,1.0,2021.0,"{""type"":""Polygon"",""coordinates"":[[[8.767845678..."
4,0_00000000000000001131,182,Nigeria,2215,Benue,191218,Oturkpo,NO,3000,Member State,1999,0.109107,1.995159,2021-01,0.395190,1.0,2021.0,"{""type"":""Polygon"",""coordinates"":[[[8.076827667..."


In [2]:
ndvi.columns

Index(['system:index', 'ADM0_CODE', 'ADM0_NAME', 'ADM1_CODE', 'ADM1_NAME',
       'ADM2_CODE', 'ADM2_NAME', 'DISP_AREA', 'EXP2_YEAR', 'STATUS',
       'STR2_YEAR', 'Shape_Area', 'Shape_Leng', 'date', 'mean', 'month',
       'year', '.geo'],
      dtype='object')

In [3]:
# drop all unrelated columns
ndvi_clean = ndvi[['date', 'mean', 'ADM2_NAME']].copy()
ndvi_clean.head()


,date,mean,ADM2_NAME
0,2021-01,0.411941,Agatu
1,2021-01,0.323571,Logo
2,2021-01,0.419388,Apa
3,2021-01,0.369389,Tarka
4,2021-01,0.395190,Oturkpo


In [4]:
# renane columns
ndvi_clean = ndvi_clean.rename(columns={
    'ADM2_NAME': 'lga',
    'mean': 'mean_ndvi'
})
ndvi_clean.columns

Index(['date', 'mean_ndvi', 'lga'], dtype='object')

In [5]:
soil.head()

,lga_name,month,soil_moisture,year
0,Agatu,1,0.216321,2021
1,Logo,1,0.151210,2021
2,Apa,1,0.199581,2021
3,Tarka,1,0.238928,2021
4,Oturkpo,1,0.209868,2021


In [6]:
# create a new date column
soil['date'] = (
    soil['year'].astype(str) + '-' +
    soil['month'].astype(str).str.zfill(2)
)

soil.head()

,lga_name,month,soil_moisture,year,date
0,Agatu,1,0.216321,2021,2021-01
1,Logo,1,0.151210,2021,2021-01
2,Apa,1,0.199581,2021,2021-01
3,Tarka,1,0.238928,2021,2021-01
4,Oturkpo,1,0.209868,2021,2021-01


In [7]:
# drop all unrelated columns
soil_clean = soil[['date', 'soil_moisture', 'lga_name']].copy()
soil_clean.head()


,date,soil_moisture,lga_name
0,2021-01,0.216321,Agatu
1,2021-01,0.151210,Logo
2,2021-01,0.199581,Apa
3,2021-01,0.238928,Tarka
4,2021-01,0.209868,Oturkpo


In [8]:
# rename the lga_name for consistency
soil_clean = soil_clean.rename(columns={'lga_name': 'lga'})
soil_clean.head()

,date,soil_moisture,lga
0,2021-01,0.216321,Agatu
1,2021-01,0.151210,Logo
2,2021-01,0.199581,Apa
3,2021-01,0.238928,Tarka
4,2021-01,0.209868,Oturkpo


In [9]:
humidity.head()

,date,lga,relative_humidity_percent
0,2021-01,Agatu,33.340072
1,2021-01,Logo,24.340416
2,2021-01,Apa,30.745435
3,2021-01,Tarka,27.835623
4,2021-01,Oturkpo,32.028395


In [10]:
# rename and reorder for consistency

humidity = humidity.rename(columns={'relative_humidity_percent': 'rel_humidity'})
humidity = humidity[['date', 'rel_humidity', 'lga']]

humidity.head()

,date,rel_humidity,lga
0,2021-01,33.340072,Agatu
1,2021-01,24.340416,Logo
2,2021-01,30.745435,Apa
3,2021-01,27.835623,Tarka
4,2021-01,32.028395,Oturkpo


In [11]:
# fix data types

for df in [ndvi_clean, soil_clean, humidity]:
    df['lga'] = df['lga'].astype(str)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
    
# Ensure numeric
ndvi_clean['mean_ndvi'] = ndvi_clean['mean_ndvi'].astype(float)
soil_clean['soil_moisture'] = soil_clean['soil_moisture'].astype(float)
humidity['rel_humidity'] = humidity['rel_humidity'].astype(float)


In [12]:
# merge as a single climatic dataset

master = ndvi_clean.merge(soil_clean, on=['date','lga'], how='outer')
master = master.merge(humidity, on=['date','lga'], how='outer')

# Quick check
print(master.head())


        date  mean_ndvi     lga  soil_moisture  rel_humidity
0 2021-01-01   0.437604     Ado       0.249529     38.163168
1 2021-01-01   0.411941   Agatu       0.216321     33.340072
2 2021-01-01   0.419388     Apa       0.199581     30.745435
3 2021-01-01   0.359085  Bukuru       0.149664     27.282734
4 2021-01-01   0.361913   Gboko       0.207243     28.148915


In [13]:
# reorder the columns
master = master[['date', 'lga', 'mean_ndvi', 'soil_moisture', 'rel_humidity']]
master.head()

,date,lga,mean_ndvi,soil_moisture,rel_humidity
0,2021-01-01,Ado,0.437604,0.249529,38.163168
1,2021-01-01,Agatu,0.411941,0.216321,33.340072
2,2021-01-01,Apa,0.419388,0.199581,30.745435
3,2021-01-01,Bukuru,0.359085,0.149664,27.282734
4,2021-01-01,Gboko,0.361913,0.207243,28.148915


In [14]:

# save data
master.to_csv('outputs/climatic_data.csv', index=False)
print("data saved successfully!")


data saved successfully!


In [ ]:
pwd()